<a href="https://colab.research.google.com/github/Puer-Hyun/Lecture-Summary/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import torch
from torch import nn

import numpy as np

모델은 단어를 입력 받아 문장에서 다음 문자가 무엇인지 예측할 것입니다. 이 과정은 원하는 길이의 문장을 생성할 때까지 반복됩니다.

간단하게 하기 위해, 큰 규모의 외부 데이터셋을 사용하지 않을 것입니다. 대신 몇 개의 문장을 정의하여 모델이 이 문장들로부터 어떻게 학습하는지 살펴볼 것입니다. 이 구현에서 진행될 과정은 다음과 같습니다:

먼저, PyTorch 패키지와 함께 모델 구축에 사용할 nn 패키지를 가져옵니다. 또한, 데이터 전처리에 NumPy만 사용할 것입니다. 왜냐하면 Torch는 NumPy와 매우 잘 작동하기 때문입니다.

첫째, 모델에 첫 단어 또는 첫 몇 개의 문자를 입력할 때 출력하려는 문장을 정의합니다.

그런 다음 문장에 있는 모든 문자로 딕셔너리를 생성하고 정수에 매핑합니다. 이를 통해 입력 문자를 해당 정수(char2int)로 변환하고 반대로 정수를 문자(int2char)로 변환할 수 있습니다.

In [52]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))
print(chars)

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))
print(int2char)

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}
print(char2int)

{'o', 'r', 'u', 'n', ' ', 'f', 'd', 'h', 'w', 'c', 'a', 'i', 'v', 'g', 'e', 'y', 'm'}
{0: 'o', 1: 'r', 2: 'u', 3: 'n', 4: ' ', 5: 'f', 6: 'd', 7: 'h', 8: 'w', 9: 'c', 10: 'a', 11: 'i', 12: 'v', 13: 'g', 14: 'e', 15: 'y', 16: 'm'}
{'o': 0, 'r': 1, 'u': 2, 'n': 3, ' ': 4, 'f': 5, 'd': 6, 'h': 7, 'w': 8, 'c': 9, 'a': 10, 'i': 11, 'v': 12, 'g': 13, 'e': 14, 'y': 15, 'm': 16}


다음으로, 모든 문장이 표준 길이를 갖도록 입력 문장에 패딩을 적용합니다. RNN은 일반적으로 가변 길이의 입력을 처리할 수 있지만, 학습 과정을 가속화하기 위해 일반적으로 배치를 사용하여 학습 데이터를 입력합니다. 데이터를 배치로 학습하기 위해서는 입력 데이터 내의 각 시퀀스가 동일한 크기를 가져야 합니다.

따라서 대부분의 경우, 길이가 너무 짧은 시퀀스는 0 값으로 채우고, 길이가 너무 긴 시퀀스는 자르는 방식으로 패딩을 수행할 수 있습니다. 우리의 경우, 가장 긴 시퀀스의 길이를 찾고 나머지 문장에 공백을 추가하여 해당 길이에 맞춥니다.

In [53]:
# Finding the length of the longest string in our data
maxlen = len(max(text, key=len))

# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence
for i in range(len(text)):
  while len(text[i])<maxlen:
      text[i] += ' '

In [54]:
[len(t) for t in text]

[15, 15, 15]

각 시간 단계에서 시퀀스의 다음 문자를 예측할 것이므로, 각 문장을 다음과 같이 나누어야 합니다:

1. 입력 데이터:
   모델에 입력할 필요가 없는 마지막 입력 문자는 제외합니다.
2. 타겟/실제 레이블:
   입력 데이터보다 한 시간 단계 앞서 있는데, 이것이 각 시간 단계에서 입력 데이터에 대한 모델의 "정답"이 됩니다.

In [55]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
  input_seq.append(text[i][:-1])
    
    # Remove first character for target sequence
  target_seq.append(text[i][1:])
  print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


In [56]:
input_seq

['hey how are yo', 'good i am fine', 'have a nice da']

입력 시퀀스와 타겟 시퀀스는 다음과 같이 표시됩니다:

입력 시퀀스: hey how are yo
타겟 시퀀스: ey how are you

타겟 시퀀스는 항상 입력 시퀀스보다 한 시간 단계 앞서 있습니다.

이제 위에서 생성한 사전을 사용하여 입력 시퀀스와 타겟 시퀀스를 문자 시퀀스가 아닌 정수 시퀀스로 변환할 수 있습니다. 이렇게 하면 입력 시퀀스를 원-핫 인코딩할 수 있습니다.

In [57]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

In [58]:
input_seq

[[7, 14, 15, 4, 7, 0, 8, 4, 10, 1, 14, 4, 15, 0],
 [13, 0, 0, 6, 4, 11, 4, 10, 16, 4, 5, 11, 3, 14],
 [7, 10, 12, 14, 4, 10, 4, 3, 11, 9, 14, 4, 6, 10]]

원-핫 벡터로 입력 시퀀스를 인코딩하기 전에, 3가지 주요 변수를 정의하겠습니다:

* dict_size: 사전 크기 - 텍스트에 있는 고유한 문자의 수
이 값은 각 문자에 할당된 인덱스를 가진 벡터이기 때문에 원-핫 벡터의 크기를 결정합니다.
* seq_len: 모델에 입력하는 시퀀스의 길이
모든 문장의 길이를 가장 긴 문장과 같게 표준화했으므로, 이 값은 최대 길이 - 1이 됩니다. 마지막 문자 입력을 제거했기 때문입니다.
* batch_size: 정의한 문장의 수로, 모델에 한 번에 입력할 배치 크기입니다.

In [59]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)
print("dict_size:", dict_size)
print("seq_len:", seq_len)
print("batch_size:", batch_size)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

dict_size: 17
seq_len: 14
batch_size: 3



우리는 각 문자에 대해 0으로 채워진 배열을 생성하고 해당 문자 인덱스를 1로 바꾸는 도우미 함수를 정의했습니다.

데이터 전처리를 모두 완료했으므로 이제 데이터를 NumPy 배열에서 PyTorch의 고유한 데이터 구조인 Torch Tensor로 옮길 수 있습니다.

이제 이 프로젝트의 재미있는 부분에 도달했습니다! Torch 라이브러리를 사용하여 모델을 정의할 것이며, 여기서 완전 연결층, 합성곱층, 기본 RNN 층, LSTM 층 등 다양한 층을 추가하거나 제거할 수 있습니다. 이 게시물에서는 기본 nn.rnn을 사용하여 RNN이 어떻게 사용될 수 있는지 간단한 예제를 보여줄 것입니다.

모델을 구축하기 전에 PyTorch의 내장 기능을 사용하여 실행 중인 장치(CPU 또는 GPU)를 확인해봅시다. 이 구현에서는 훈련이 매우 간단하기 때문에 GPU가 필요하지 않습니다. 그러나 대규모 데이터셋과 수백만 개의 훈련 가능한 매개변수가 있는 모델로 진행하게 되면, 훈련 속도를 높이기 위해 GPU를 사용하는 것이 매우 중요해집니다.

In [60]:
input_seq

[[7, 14, 15, 4, 7, 0, 8, 4, 10, 1, 14, 4, 15, 0],
 [13, 0, 0, 6, 4, 11, 4, 10, 16, 4, 5, 11, 3, 14],
 [7, 10, 12, 14, 4, 10, 4, 3, 11, 9, 14, 4, 6, 10]]

In [61]:
# 각각의 문장이 알파벳 별로 16,4,10 과 같이 표현이 되었는데, 이것을 다시 원핫인코딩을 해주느라 
# hey에서 e라는 알파벳은 기존에는 4 였지만, 바뀐 벡터에선 [0,0,0,0, 1,0,0,0,,,,,, 0] = 17차원 짜리로 변경되었다.
# Input shape --> (Batch Size, Sequence Length, One-Hot Encoding Size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [62]:
input_seq.shape 
# 14인 이유는 각 시퀀스가 14글자로 이루어져 있고, 17인 이유는 사전에 표현된 알파벳 개수가 전체 17개이기 때문

(3, 14, 17)

In [63]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [64]:
input_seq = torch.from_numpy(input_seq)
input_seq = input_seq.to(device)
target_seq = torch.Tensor(target_seq).to(device)

In [65]:
target_seq

tensor([[14., 15.,  4.,  7.,  0.,  8.,  4., 10.,  1., 14.,  4., 15.,  0.,  2.],
        [ 0.,  0.,  6.,  4., 11.,  4., 10., 16.,  4.,  5., 11.,  3., 14.,  4.],
        [10., 12., 14.,  4., 10.,  4.,  3., 11.,  9., 14.,  4.,  6., 10., 15.]],
       device='cuda:0')

In [66]:
input_seq

tensor([[[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

우리만의 신경망 모델을 구축하기 시작하려면, 모든 신경망 모듈에 대한 PyTorch의 기본 클래스(nn.module)를 상속하는 클래스를 정의할 수 있습니다. 그렇게 하면 생성자 아래에 모델에 대한 변수와 레이어를 정의할 수 있습니다. 이 모델에서는 RNN 1개 층과 전결합층만 사용할 것입니다. 전결합층은 RNN 출력을 원하는 출력 형태로 변환하는 역할을 담당합니다.

또한 클래스 메소드로 forward() 아래에 순방향 전달 함수를 정의해야 합니다. 순방향 함수는 순차적으로 실행되므로, RNN 출력을 완전 연결된 층에 전달하기 전에 먼저 입력과 0으로 초기화된 은닉 상태를 RNN 층에 전달해야 합니다. 생성자에서 정의한 레이어를 사용하고 있음을 주의하세요.

마지막으로 정의해야 할 메소드는 이전에 은닉 상태를 초기화하기 위해 호출한 메소드인 init_hidden()입니다. 이 메소드는 은닉 상태 모양의 0으로 채워진 텐서를 생성합니다.

In [67]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden.to(device)

위에서 모델을 정의한 후, 관련 매개변수를 사용하여 모델을 인스턴스화하고 하이퍼파라미터를 정의해야 합니다. 아래에서 정의하는 하이퍼파라미터는 다음과 같습니다:

n_epochs: 에포크 수 --> 모델이 전체 학습 데이터셋을 몇 번 반복할지 결정
lr: 학습률 --> 모델이 역전파가 수행될 때마다 셀의 가중치를 업데이트하는 속도
하이퍼파라미터에 대한 보다 깊이 있는 가이드를 원하신다면, 이 포괄적인 글을 참고하십시오.

다른 신경망과 마찬가지로, 최적화기와 손실 함수도 정의해야 합니다. 최종 출력이 기본적으로 분류 작업이므로 CrossEntropyLoss를 사용하고 일반적인 Adam 최적화기를 사용할 것입니다.

In [68]:
print(dict_size)

17


In [69]:
print(device)

cuda


In [70]:
# Instantiate the model with hyperparameters
model = Model(input_size = dict_size, output_size = dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [71]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.4524
Epoch: 20/100............. Loss: 2.1997
Epoch: 30/100............. Loss: 1.8606
Epoch: 40/100............. Loss: 1.4217
Epoch: 50/100............. Loss: 0.9857
Epoch: 60/100............. Loss: 0.6517
Epoch: 70/100............. Loss: 0.4256
Epoch: 80/100............. Loss: 0.2864
Epoch: 90/100............. Loss: 0.2044
Epoch: 100/100............. Loss: 0.1554


In [75]:
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [76]:
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [77]:
sample(model, 15, 'good')

'good i am fine '

In [ ]:
sample(model, 30, 'hi')

'hive a nice day hive a nice da'